In [117]:
import csv
import re
import json
import os
import threading
from pprint import pprint as pp
from functools import reduce
import itertools as it
import pandas as pd

In [2]:
from pythainlp.sentiment import sentiment
import pandas

In [3]:
user_tweet_files = os.listdir('./user-tweet')
print('# of user file = %d' % len(user_tweet_files))

# of user file = 4672


In [9]:
party_name = ['ประชาธิปัตย์', 'พลังประชารัฐ', 'รวมพลังประชาชาติไทย', 'เสรีรวมไทย', 'เพื่อไทย', 'อนาคตใหม่']
user_sentiment_list = []

In [15]:
def sentiment_thread(tweet_data, index):
    if index % 10 == 0:
        print('Now, the index is %d' % index)
        
    user_sentiment = {
        'ประชาธิปัตย์': {'pos': 0, 'neg': 0}, 
        'พลังประชารัฐ': {'pos': 0, 'neg': 0}, 
        'รวมพลังประชาชาติไทย': {'pos': 0, 'neg': 0}, 
        'เสรีรวมไทย': {'pos': 0, 'neg': 0}, 
        'เพื่อไทย': {'pos': 0, 'neg': 0}, 
        'อนาคตใหม่':{'pos': 0, 'neg': 0},
    }
    
    for datum in tweet_data:
        tweet = datum['text']
        
        if not any([party in tweet for party in party_name]):
            continue
        
        result = sentiment(tweet)
        for party in party_name:
            if party in tweet:
                user_sentiment[party][result] += 1

    user_sentiment_list.append(user_sentiment)
    

In [16]:
thd_list = []

for index in range(0, len(user_tweet_files)):    
    file = user_tweet_files[index]
    path_to_file = './user-tweet/%s' % file
    user = file.split('.')[0]
    
    with open(path_to_file, 'r', encoding='utf8') as fp:
        data = json.load(fp)
    fp.close()    
    
    thd = threading.Thread(target=sentiment_thread, args=(data,index))
    thd_list.append(thd)

for thd in thd_list:
    thd.start()
    thd.join()
    
print(len(user_sentiment_list))

Now, the index is 0
Now, the index is 5
Now, the index is 10
Now, the index is 15
Now, the index is 20
Now, the index is 25
Now, the index is 30
Now, the index is 35
Now, the index is 40
Now, the index is 45
Now, the index is 50
Now, the index is 55
Now, the index is 60
Now, the index is 65
Now, the index is 70
Now, the index is 75
Now, the index is 80
Now, the index is 85
Now, the index is 90
Now, the index is 95
Now, the index is 100
Now, the index is 105
Now, the index is 110
Now, the index is 115
Now, the index is 120
Now, the index is 125
Now, the index is 130
Now, the index is 135
Now, the index is 140
Now, the index is 145
Now, the index is 150
Now, the index is 155
Now, the index is 160
Now, the index is 165
Now, the index is 170
Now, the index is 175
Now, the index is 180
Now, the index is 185
Now, the index is 190
Now, the index is 195
Now, the index is 200
Now, the index is 205
Now, the index is 210
Now, the index is 215
Now, the index is 220
Now, the index is 225
Now, the i

Now, the index is 1825
Now, the index is 1830
Now, the index is 1835
Now, the index is 1840
Now, the index is 1845
Now, the index is 1850
Now, the index is 1855
Now, the index is 1860
Now, the index is 1865
Now, the index is 1870
Now, the index is 1875
Now, the index is 1880
Now, the index is 1885
Now, the index is 1890
Now, the index is 1895
Now, the index is 1900
Now, the index is 1905
Now, the index is 1910
Now, the index is 1915
Now, the index is 1920
Now, the index is 1925
Now, the index is 1930
Now, the index is 1935
Now, the index is 1940
Now, the index is 1945
Now, the index is 1950
Now, the index is 1955
Now, the index is 1960
Now, the index is 1965
Now, the index is 1970
Now, the index is 1975
Now, the index is 1980
Now, the index is 1985
Now, the index is 1990
Now, the index is 1995
Now, the index is 2000
Now, the index is 2005
Now, the index is 2010
Now, the index is 2015
Now, the index is 2020
Now, the index is 2025
Now, the index is 2030
Now, the index is 2035
Now, the in

Now, the index is 3605
Now, the index is 3610
Now, the index is 3615
Now, the index is 3620
Now, the index is 3625
Now, the index is 3630
Now, the index is 3635
Now, the index is 3640
Now, the index is 3645
Now, the index is 3650
Now, the index is 3655
Now, the index is 3660
Now, the index is 3665
Now, the index is 3670
Now, the index is 3675
Now, the index is 3680
Now, the index is 3685
Now, the index is 3690
Now, the index is 3695
Now, the index is 3700
Now, the index is 3705
Now, the index is 3710
Now, the index is 3715
Now, the index is 3720
Now, the index is 3725
Now, the index is 3730
Now, the index is 3735
Now, the index is 3740
Now, the index is 3745
Now, the index is 3750
Now, the index is 3755
Now, the index is 3760
Now, the index is 3765
Now, the index is 3770
Now, the index is 3775
Now, the index is 3780
Now, the index is 3785
Now, the index is 3790
Now, the index is 3795
Now, the index is 3800
Now, the index is 3805
Now, the index is 3810
Now, the index is 3815
Now, the in

In [20]:
no_zero_sentiment = list(
    filter(
        lambda x: any([x[key]['pos'] != 0 or x[key]['neg'] != 0 for key in party_name]), 
        user_sentiment_list
    )
)

In [80]:
pro_party = {
    'ประชาธิปัตย์': [], 
    'พลังประชารัฐ': [], 
    'รวมพลังประชาชาติไทย': [], 
    'เสรีรวมไทย': [], 
    'เพื่อไทย': [], 
    'อนาคตใหม่': [],
}

for i in range(len(no_zero_sentiment)):
    max_pos_list = [x['pos'] for x in no_zero_sentiment[i].values()]
    max_index = max_pos_list.index(max(max_pos_list)) if sum(max_pos_list) > 1 else None
    if max_index != None:
        max_pos_party = list(pro_party.keys())[max_index]
        pro_party[max_pos_party].append(no_zero_sentiment[i])

pp(pro_party['อนาคตใหม่'][:2])

[{'ประชาธิปัตย์': {'neg': 0, 'pos': 0},
  'พลังประชารัฐ': {'neg': 0, 'pos': 0},
  'รวมพลังประชาชาติไทย': {'neg': 0, 'pos': 0},
  'อนาคตใหม่': {'neg': 0, 'pos': 3},
  'เพื่อไทย': {'neg': 0, 'pos': 0},
  'เสรีรวมไทย': {'neg': 0, 'pos': 0}},
 {'ประชาธิปัตย์': {'neg': 0, 'pos': 0},
  'พลังประชารัฐ': {'neg': 0, 'pos': 0},
  'รวมพลังประชาชาติไทย': {'neg': 0, 'pos': 0},
  'อนาคตใหม่': {'neg': 0, 'pos': 2},
  'เพื่อไทย': {'neg': 0, 'pos': 0},
  'เสรีรวมไทย': {'neg': 0, 'pos': 0}}]


In [123]:
def sum_neg_pos(x, y):
    return {
        'neg': x['neg'] + y['neg'],
        'pos': x['pos'] + y['pos']
    }

def sum_sentiment_dict(x, y):
    return {
        'ประชาธิปัตย์': sum_neg_pos(x['ประชาธิปัตย์'], y['ประชาธิปัตย์']),
        'พลังประชารัฐ': sum_neg_pos(x['พลังประชารัฐ'], y['พลังประชารัฐ']),
        'รวมพลังประชาชาติไทย': sum_neg_pos(x['รวมพลังประชาชาติไทย'], y['รวมพลังประชาชาติไทย']),
        'อนาคตใหม่': sum_neg_pos(x['อนาคตใหม่'], y['อนาคตใหม่']),
        'เพื่อไทย': sum_neg_pos(x['เพื่อไทย'], y['เพื่อไทย']),
        'เสรีรวมไทย': sum_neg_pos(x['เสรีรวมไทย'], y['เสรีรวมไทย'])
    }

sum_sentiment = {}
for party in pro_party:
    sum_sentiment['pro-'+party] = dict(reduce(sum_sentiment_dict, pro_party[party]))

In [150]:
df = pd.DataFrame(sum_sentiment['pro-ประชาธิปัตย์']).transpose()
df.str.encode('utf8').str.decode('utf8')
df.plot(kind='bar', stacked=True)

AttributeError: 'DataFrame' object has no attribute 'str'